In [43]:
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
from random import sample
import argparse
import numpy as np
import os
import pickle
from tqdm import tqdm
from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import mahalanobis
from scipy.ndimage import gaussian_filter
from skimage import morphology
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import matplotlib

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.models import wide_resnet50_2, resnet18

import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [44]:
class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, train=True):
        self.data = pd.read_csv(csv_file)  # Load CSV file
        self.root_dir = root_dir           # Directory where images are stored
        self.transform = transform
        self.train = train
        
        # Create a label-to-integer mapping
        self.label_map = {label: idx for idx, label in enumerate(self.data.iloc[:, -1].unique())}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Load image
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 1])  # 2nd column is filename
        image = Image.open(img_name).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        if self.train:
            label = self.data.iloc[idx, -1]  # Last column is the label
            
            # Convert the text label to an integer label
            label = self.label_map.get(label, -1)  # -1 if the label is not found, you can handle it differently
            label = torch.tensor(label, dtype=torch.long)  # Convert to tensor
            return image, label
        else:
            return image
    

In [66]:
class ImageDatasetFilename(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, train=True):
        self.data = pd.read_csv(csv_file)  # Load CSV file
        self.root_dir = root_dir           # Directory where images are stored
        self.transform = transform
        self.train = train
        
        # Create a label-to-integer mapping
        self.label_map = {label: idx for idx, label in enumerate(self.data.iloc[:, -1].unique())}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Load image
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 1])  # 2nd column is filename
        image = Image.open(img_name).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        if self.train:
            label = self.data.iloc[idx, -1]  # Last column is the label
            
            # Convert the text label to an integer label
            label = self.label_map.get(label, -1)  # -1 if the label is not found, you can handle it differently
            label = torch.tensor(label, dtype=torch.long)  # Convert to tensor
            return image, label
        else:
            return image,img_name
    

In [45]:
# Create a new train file with just two classes
filepath = "/Users/nicolasthiou/Desktop/DataChallenge/Y_train_eVW9jym.csv"
train_dataset = pd.read_csv(filepath)
train_dataset = train_dataset.replace("GOOD", "nm")
train_dataset =train_dataset.replace("Boucle plate", "nm")
train_dataset =train_dataset.replace("Lift-off noir", "nm")
train_dataset =train_dataset.replace("Lift-off blanc", "nm")
train_dataset =train_dataset.replace("Short circuit MOS", "nm")

train_dataset.to_csv("modified_train.csv")


In [46]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
])

In [47]:
train_dataset = ImageDataset(csv_file="/Users/nicolasthiou/Desktop/DataChallenge/Y_train_eVW9jym.csv", root_dir="/Users/nicolasthiou/Desktop/DataChallenge/preprocessed_train", transform=transform, train=True)
test_dataset = ImageDataset(csv_file="/Users/nicolasthiou/Desktop/DataChallenge/Y_random_nKwalR1.csv", root_dir="/Users/nicolasthiou/Desktop/DataChallenge/preprocessed_test", transform=transform, train=False)


Classifier 1, decides if missing or not missing

In [49]:
#Train classifier 1

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

import torchvision.models as models
import torch.nn as nn

# Load Pretrained ResNet50
model = models.resnet50(pretrained=True)

num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)

import torch.optim as optim

# Move model to GPU if available
device = torch.device("mps")
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")

/Users/nicolasthiou/miniconda3/envs/STMC/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nicolasthiou/miniconda3/envs/STMC/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 259/259 [01:08<00:00,  3.78it/s]


Epoch [1/20], Loss: 0.0125


100%|██████████| 259/259 [01:09<00:00,  3.75it/s]


Epoch [2/20], Loss: 0.0069


100%|██████████| 259/259 [01:10<00:00,  3.69it/s]


...

Epoch [18/20], Loss: 0.0028


100%|██████████| 259/259 [01:13<00:00,  3.53it/s]


Epoch [19/20], Loss: 0.0036


100%|██████████| 259/259 [01:11<00:00,  3.62it/s]

Epoch [20/20], Loss: 0.0031
Training complete!


In [50]:
# Cell 19: Save the model
# Save the trained model
model_name = 'MissingOrNot'
model_save_path = 'MissingOrNot.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'class_mapping': train_dataset.label_map,
    'model_name': model_name,
}, model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to MissingOrNot.pth


In [95]:
classifier_file1 = '/Users/nicolasthiou/Desktop/DataChallenge/data_challenge/MissingOrNot.pth'
image_folder = '/Users/nicolasthiou/Desktop/DataChallenge/preprocessed_test'

# Transformation des images pour l'entrée du modèle
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
])

# Chargement du dataset
dataset = ImageDatasetFilename(csv_file="/Users/nicolasthiou/Desktop/DataChallenge/Y_random_nKwalR1.csv", root_dir="/Users/nicolasthiou/Desktop/DataChallenge/preprocessed_test", transform=transform, train=False)

dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# device setup
use_cuda = torch.cuda.is_available()
device = torch.device('mps')
num_classes = 2  # Set this to match your training setup
model = models.resnet50()
model.fc = torch.nn.Linear(in_features=2048, out_features=num_classes)  # Match trained model

# Load the saved dictionary
checkpoint = torch.load(classifier_file1, map_location=device)

# Load model state dict
model.load_state_dict(checkpoint['model_state_dict'])  
model.to(device)
model.eval()

predictions = []
probabilities = []

with torch.no_grad():  # Disable gradient calculation for inference
    for images in test_loader:
        images = images.to(device)
        
        # Get model outputs
        outputs = model(images)
        
        # Apply softmax to get class probabilities
        probs = F.softmax(outputs, dim=1)  # probs is a tensor with shape [batch_size, num_classes]
        
        # Get predicted class (index of maximum probability)
        _, preds = torch.max(probs, 1)  # Get the class index with highest probability

        predictions.extend(preds.cpu().numpy())  # Convert predictions to numpy and append
        probabilities.extend(probs.cpu().numpy())  # Append probabilities as well

# Create a DataFrame with the results
import pandas as pd

test_filenames = pd.read_csv("/Users/nicolasthiou/Desktop/DataChallenge/Y_random_nKwalR1.csv").iloc[:, 1]  # Get filenames from the test CSV

# Create a DataFrame for predictions with probabilities
df = pd.DataFrame({
    "filename": test_filenames,
    "label": predictions,
    "probabilities": [prob.tolist()[0] for prob in probabilities]  # Convert each prob tensor to a list
})

# Save to CSV
df.to_csv("PRED1.csv", index=False)

print("Predictions and probabilities saved")

/var/folders/k8/nrjdn0dn3s56bt4jf41m2rcc0000gn/T/ipykernel_44638/2775736986.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(classifier_file1, ma

Predictions and probabilities saved


Classifier 2, decides the class if not missing

In [93]:
classifier_file2 = '/Users/nicolasthiou/Desktop/DataChallenge/data_challenge/image_classifier_model_bis.pth'
image_folder = '/Users/nicolasthiou/Desktop/DataChallenge/preprocessed_test'

# Transformation des images pour l'entrée du modèle
transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
])

# Chargement du dataset
dataset = ImageDatasetFilename(csv_file="/Users/nicolasthiou/Desktop/DataChallenge/Y_random_nKwalR1.csv", root_dir="/Users/nicolasthiou/Desktop/DataChallenge/preprocessed_test", transform=transform, train=False)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# device setup
use_cuda = torch.cuda.is_available()
device = torch.device('mps')
num_classes = 6  # Set this to match your training setup
model = models.resnet50()
model.fc = torch.nn.Linear(in_features=2048, out_features=num_classes)  # Match trained model

# Load the saved dictionary
checkpoint = torch.load(classifier_file2, map_location=device)

# Load model state dict
model.load_state_dict(checkpoint['model_state_dict'])  
model.to(device)
model.eval()

predictions = []
probabilities = []

with torch.no_grad():  # Disable gradient calculation for inference
    for images in test_loader:
        images = images.to(device)
        
        # Get model outputs
        outputs = model(images)
        
        # Apply softmax to get class probabilities
        probs = F.softmax(outputs, dim=1)  # probs is a tensor with shape [batch_size, num_classes]
        
        # Get predicted class (index of maximum probability)
        _, preds = torch.max(probs, 1)  # Get the class index with highest probability

        predictions.extend(preds.cpu().numpy())  # Convert predictions to numpy and append
        probabilities.extend(probs.cpu().numpy())  # Append probabilities as well

# Create a DataFrame with the results
import pandas as pd

test_filenames = pd.read_csv("/Users/nicolasthiou/Desktop/DataChallenge/Y_random_nKwalR1.csv").iloc[:, 1]  # Get filenames from the test CSV

# Create a DataFrame for predictions with probabilities
df = pd.DataFrame({
    "filename": test_filenames,
    "label": predictions,
    "probabilities": [prob.tolist() for prob in probabilities]  # Convert each prob tensor to a list
})

# Save to CSV
df.to_csv("PRED2.csv", index=False)

print("Predictions and probabilities saved")

/var/folders/k8/nrjdn0dn3s56bt4jf41m2rcc0000gn/T/ipykernel_44638/1511014312.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(classifier_file2, ma

Predictions and probabilities saved


Anomaly detector, again only if not missing

In [71]:
test_path =  '/Users/nicolasthiou/Desktop/DataChallenge/preprocessed_test'
padim_trained = '/Users/nicolasthiou/Desktop/DataChallenge/models_GX6qjjM/PADIM.pkl'

class CustomDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.images = []


        for img_name in os.listdir(self.data_path):
            img_path = os.path.join(self.data_path, img_name)
            self.images.append(img_path)


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")  # Charger l'image
    
        # Appliquer les transformations
        if self.transform:
            image = self.transform(image)  # Appliquer les transformations à l'image

        filename = os.path.basename(img_path)

        return filename, image                               


def denormalization(x):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    x = (((x.transpose(1, 2, 0) * std) + mean) * 255.).astype(np.uint8)
    
    return x


def embedding_concat(x, y):
    B, C1, H1, W1 = x.size()
    _, C2, H2, W2 = y.size()
    s = int(H1 / H2)
    x = F.unfold(x, kernel_size=s, dilation=1, stride=s)
    x = x.view(B, C1, -1, H2, W2)
    z = torch.zeros(B, C1 + C2, x.size(2), H2, W2)
    for i in range(x.size(2)):
        z[:, :, i, :, :] = torch.cat((x[:, :, i, :, :], y), 1)
    z = z.view(B, -1, H2 * W2)
    z = F.fold(z, kernel_size=s, output_size=(H1, W1), stride=s)

    return z


# Charger les données d'entraînement
with open(padim_trained, 'rb') as f:
    train_outputs = pickle.load(f)

model = wide_resnet50_2(pretrained=True, progress=True)
t_d = 1792
d = 550


# device setup
use_cuda = torch.cuda.is_available()
device = torch.device('mps')

model.to(device)
model.eval()
random.seed(1024)
torch.manual_seed(1024)
if use_cuda:
    torch.cuda.manual_seed_all(1024)

idx = torch.tensor(sample(range(0, t_d), d))

# set model's intermediate outputs
outputs = []

def hook(module, input, output):
    outputs.append(output)

model.layer1[-1].register_forward_hook(hook)
model.layer2[-1].register_forward_hook(hook)
model.layer3[-1].register_forward_hook(hook)


transform = transforms.Compose([
                                transforms.Resize((128, 128)),  # Redimensionner les images
                                transforms.ToTensor(),            # Convertir en tenseur
                                ])

test_dataset = CustomDataset(test_path, transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, pin_memory=True)
test_outputs = OrderedDict([('layer1', []), ('layer2', []), ('layer3', [])])

test_imgs = []
filenames_list = []
scores_list = []

# extract test set features
for filename, x in tqdm(test_dataloader, '| feature extraction | test |'):
    filenames_list.extend(filename)  # Collect filenames for the batch
    test_imgs.extend(x.cpu().detach().numpy())
    # model prediction
    with torch.no_grad():
        _ = model(x.to(device))
    # get intermediate layer outputs
    for k, v in zip(test_outputs.keys(), outputs):
        test_outputs[k].append(v.cpu().detach())
    # initialize hook outputs
    outputs = []
for k, v in test_outputs.items():
    test_outputs[k] = torch.cat(v, 0)

# Embedding concat
embedding_vectors = test_outputs['layer1']
for layer_name in ['layer2', 'layer3']:
    embedding_vectors = embedding_concat(embedding_vectors, test_outputs[layer_name])

# randomly select d dimension
embedding_vectors = torch.index_select(embedding_vectors, 1, idx)

# calculate distance matrix
B, C, H, W = embedding_vectors.size()
embedding_vectors = embedding_vectors.view(B, C, H * W).numpy()
dist_list = []
for i in range(H * W):
    mean = train_outputs[0][:, i]
    conv_inv = np.linalg.inv(train_outputs[1][:, :, i])
    dist = [mahalanobis(sample[:, i], mean, conv_inv) for sample in embedding_vectors]
    dist_list.append(dist)

dist_list = np.array(dist_list).transpose(1, 0).reshape(B, H, W)

# upsample
dist_list = torch.tensor(dist_list)
score_map = F.interpolate(dist_list.unsqueeze(1), size=x.size(2), mode='bilinear',
                            align_corners=False).squeeze().numpy()

# apply gaussian smoothing on the score map
for i in range(score_map.shape[0]):
    score_map[i] = gaussian_filter(score_map[i], sigma=4)

# Normalization
max_score = score_map.max()
min_score = score_map.min()
scores = (score_map - min_score) / (max_score - min_score)

# calculate image-level ROC AUC score
img_scores = scores.reshape(scores.shape[0], -1).max(axis=1)

# Store scores and filenames
for filename, score in zip(filenames_list, img_scores):
    scores_list.append((filename, score))
    
y_pred_anomaly = scores_list

/Users/nicolasthiou/miniconda3/envs/STMC/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nicolasthiou/miniconda3/envs/STMC/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet50_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet50_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
| feature extraction | test |: 100%|██████████| 33/33 [00:02<00:00, 11.23it/s]


Decision process


In [106]:
import numpy as np

# Assuming y_pred_anomaly is a list of tuples like [(filename1, prob1), (filename2, prob2), ...]
anomaly_dict = {filename: prob for filename, prob in y_pred_anomaly}

df1 = pd.read_csv("/Users/nicolasthiou/Desktop/DataChallenge/data_challenge/PRED1.csv")
df2 = pd.read_csv("/Users/nicolasthiou/Desktop/DataChallenge/data_challenge/PRED2.csv")
test_df = pd.read_csv('/Users/nicolasthiou/Desktop/DataChallenge/Y_random_nKwalR1.csv')

new_rows = []
threshold = 0.5

# Loop through each row in df1
for i in range(len(df1)):
    filename = df1.iloc[i]['filename']  # Get the filename from df1
    p_missing = df1.iloc[i]['probabilities']  # Probability of missing from df1
    p_drift = anomaly_dict.get(filename, 0)  # Access p_drift from the anomaly_dict based on filename
    
    # Get the string of probabilities and clean it up (remove brackets and split by commas)
    p_classes_str = df2.iloc[i]['probabilities']  # Class probabilities from df2
    p_classes_str = p_classes_str.strip('[]')  # Remove the brackets
    p_classes = np.array([float(x) for x in p_classes_str.split(',')])  # Convert to list of floats

    # Find the row in test_df that matches the filename
    test_row = test_df[test_df['filename'] == filename]
    if not test_row.empty:
        window = test_row.iloc[0]['window']  # Extract 'window' value
        lib = test_row.iloc[0]['lib']  # Extract 'lib' value

        # Decide the label based on the conditions
        if p_missing > 0.9:
            label = 4
        elif p_drift > threshold:
            label = 6
        else:
            label = np.argmax(p_classes)  # Use argmax to get the index of the max

        # Append the new row with the results
        new_rows.append([i, filename, window, lib, label])

# Create a pandas DataFrame with the results
final_df = pd.DataFrame(new_rows, columns=['', 'filename', 'window', 'lib', 'Label'])

# Save the final CSV
final_df.to_csv('Final_grosse_archi.csv', index=False)
